<!-- <div  style="background-color:#FA8072" class="header"> -->
<div class="header">
  <h1>Tutorial</h1>
  <p>A step-by-step guidance to DrMD package.</p>
</div>


We first need to import relevant classes.

The circuit in this backacge is a 2-qubit circuit. 

In [86]:
from qubit_state import QubitState
from unitary_gate import UnitaryGate as ug
import gate_list as gl
from circuit import Circuit as circ
# import numpy as np
# import matplotlib.pyplot as plt

<h2> Qubit States </h2>

We can create a new qubit state using the matrix representation with <code>QubitState()</code>. Let the first qubit be in the $|0\rangle$ state while the second qubit in the $|1\rangle$ - this corresponds to the matrix $[0, 1, 0, 0]$. Otherwise, we can also create this qubit by inputing two single-qubit matrices $[1,0], [0,1]$.

In [3]:
qb1 = QubitState([0,1,0,0])
qb2 = QubitState([1,0], (0,1))

Using the <code>peek()</code> function, we can check that these two states are the same.

In [4]:
print(qb1.peek())
print(qb2.peek())

[0 1 0 0]
[0 1 0 0]


<h3>Measure a Qubit State</h3>

 With a qubit in hand (?), the simplest thing one can do is to measure it. With <code>DrMD</code>, there are two ways of measuring a qubit.
 1. Using <code>measureCollapse()</code> function which mimics the collapse of the state upon being measured. This returns an eigenstate.
 2. Using <code>measureStats()</code> function which does not collapse the state. Instead, this function will return the probability with which the initial state collapses to a certain eigenstate.

For example, let initialise our qubits in the state $|\psi_0\rangle = \frac{1}{2}(|00\rangle + |10\rangle + |01\rangle + |11\rangle)$.

We see that <code>measureCollapse()</code> collapes the state $|\psi_0\rangle$ into the state $|00\rangle$ (or $|10\rangle, |01\rangle, |11\rangle$) with a 25% probability.


In [5]:
qb = QubitState([1, 1, 1, 1])
print(qb.peek())
qb.measureCollapse()
print(qb.peek())

[0.5 0.5 0.5 0.5]
[0 0 0 1]


Moreover, we can use <code>to_measure</code> parameter to specify the qubit on which we are doing partial measurement. For example, in this case, we are doing partial measurement on the first qubit. We see that the function returns the state $|0 0 1 1\rangle$ (or $|1 1 0 0\rangle$) 50% of the time.

In [6]:
qb = QubitState([1, 1, 1, 1])
print(qb.peek())
qb.measureCollapse(to_measure=1)
print(qb.peek())

[0.5 0.5 0.5 0.5]
[0.70710678 0.70710678 0.         0.        ]


We see that <code>measureStats()</code> returns an array indicating which state and with what probability our initial $|\psi_0\rangle$ would collapse to, without actually collapsing it.

In [7]:
qb = QubitState([1,1,1,1])
print(qb.measureStats())
print(qb.measureStats(to_measure=1))


[([1 0 0 0], 0.25), ([0 1 0 0], 0.25), ([0 0 1 0], 0.25), ([0 0 0 1], 0.25)]
[([0.7071 0.7071 0.     0.    ], 0.5), ([0.     0.     0.7071 0.7071], 0.5)]


<h2>Unitary Gates</h2>

What better fun can one have with qubits than applying unitary gates to them? Nothing! That's why package <code>DrMD</code> is here to provide you nerds with the greatest fun of your life with UnitaryGate class.

One can create new unitary gates using their matrix representation, again with a 4 x 4 matrix or with two 2 x 2 matrices, each denoting a gate on a qubit. Let's try to create a $X_1Z_2$ gate and see its action on the state $|00\rangle$.

In [8]:
x1z2 = ug([[0, 1],[1, 0]], [[1, 0], [0, -1]])

In [23]:
qb00 = QubitState([1,0,0,0])
qb = x1z2.apply(qb00)
qb

[0 0 1 0]

If one is too lazy, one can also use our predefined list of Pauli gates. For example, to create $X_1Z_2$ gate, we can do as follows.

In [24]:
mat = ug(gl.X_mat, gl.Z_mat)
print(x1z2.compare(mat))

True


Apply a wrong gate and want to ctrl-z? Fret not! Use <code>dagger()</code> to apply the inverse gate.

In [25]:
x1z2.dagger().apply(qb)

[1 0 0 0]

<h2>Circuit</h2>

Let's take a step further! A curious mind may ask: "What happens if one has many gates?" The answer is what we are looking at in this section - a circuit!

In [26]:
c = circ([x1z2, gl.hadamard1])


Let see what this circuit does to our old friend $|00\rangle$.

In [27]:
c.apply(qb00)

[ 0.7071  0.     -0.7071  0.    ]

It returns $\frac{1}{\sqrt{2}}(|00\rangle - |10\rangle)$, as expected. There are many other things we can do to an existing circuit. We can <code>append</code> to add or <code>pop</code> to remove the last element, and many more. Let's try them out by preparing an entangled state. We can start from $|00\rangle$ and edit our current circuit.

In [38]:
c = circ([x1z2, gl.hadamard1])
c.insert(1,gl.y2)
c.append(gl.cnot)
c_extra = circ([gl.hadamard1, gl.x2, gl.hadamard2])
c.merge(c_extra)
c.pop()
c.isEmpty()
print(c.size())
print(c)

6
The gates applied are: 
[[ 0  0  1  0]
 [ 0  0  0 -1]
 [ 1  0  0  0]
 [ 0 -1  0  0]], 
[[0.+0.j 0.-1.j 0.+0.j 0.-0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]], 
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678 -0.        ]
 [ 0.          0.70710678 -0.         -0.70710678]], 
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]], 
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678 -0.        ]
 [ 0.          0.70710678 -0.         -0.70710678]], 
[[0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]].



In [85]:
# # qb_final = 
# res = [c.apply(qb00).measureCollapse().peek() for i in range (100)]

# #  bins = [np.array([1,0,0,0]), np.array([0,1,0,0]), np.array([0,0,1,0]), np.array([0,0,0,1])]
# np.histogram(res)
# res
